# 골프장 이용후기 데이터 수집


수집 범위  
> 골프장명, 작성자, 작성일, 평점(전체 평점, 캐디서비스, 코스관리, 가격만족도, 부대시설/식사, 숙박), 이용후기

[xgolf 이용후기 링크]
https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=1&jv_total_data=10||)

In [4]:

import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리
import re
import chromedriver_autoinstaller
from bs4 import BeautifulSoup    # html 데이터를 전처리
import requests
from selenium import webdriver   # 웹 브라우저 자동화
from urllib.request import urlopen
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm, tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.
from IPython.display import clear_output

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install chromedriver_autoinstaller

In [5]:
options = webdriver.ChromeOptions()
chrome_path = chromedriver_autoinstaller.install()

In [6]:
driver = webdriver.Chrome(chrome_path, options=options)
# 사이트 주소는 x-golf
driver.get('https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=1&jv_total_data=10||')
time.sleep(2)  # 2초간 정지

In [7]:
# '지역' 클릭
driver.find_element_by_xpath("""//*[@id="pr_area_code"]""").click( )
time.sleep(1)

# '경기도' 클릭
# 옵션넘버로 '지역 선택 가능'
# [1]전체/[2]경기/[3]인천/[4]대전/[5]충북/[6]충남/[7]강원/[8]전북/[9]전남/[10]경북/[11]경남/[12]울산/[13]대구/[14]부산/[15]제주
driver.find_element_by_xpath("""//*[@id="pr_area_code"]/option[2]""").click( )
time.sleep(1)

# '검색' 클릭
driver.find_element_by_xpath("""//*[@id="epilogueSearchForm"]/a""").click()
time.sleep(1)

In [8]:
title_list = []

# title_raw 크롤링 시작
titles = "#contents > table > tbody > tr > td.title > div > a"
title_raw = driver.find_elements_by_css_selector(titles)
title_raw[0].text

url_list = []

url = "#contents > table > tbody > tr > td.title > div > a"
url_list = driver.find_elements_by_css_selector(url)
url_list[0].get_attribute('href')

print('제목:',title_raw[0].text)
print('제목별 url:',url_list[0].get_attribute('href'))

제목: 라운드 후~~ 좋은 근무자분~~
제목별 url: https://www.xgolf.com/board/afternote/view.asp?pr_board_code=17000232&pr_gotopage=1&jv_total_data=031||


In [9]:
# 추출 사이트의 최대페이지 수
final_page = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/a[12]""")
final_pnums = final_page.get_attribute('href')
final_pnums = re.findall(r'\d+', final_pnums)[0]

print('최대 수집 가능 page수:', final_pnums,'개')


최대 수집 가능 page수: 308 개


In [10]:
# 제목 및 url 추출(href 속성값)
title_list = []

for j in tqdm(title_raw):
    title = j.text
    title_list.append(title)
    # print(title)
    

url_list = []
url = "#contents > table > tbody > tr > td.title > div > a"
links = driver.find_elements_by_css_selector(url)
for k in tqdm(links):
    url_list.append(k.get_attribute('href'))
    # print(i.get_attribute('href'))


clear_output()    
print("")
print('title갯수: ', len(title_list))
print('url갯수: ', len(url_list))


title갯수:  500
url갯수:  500


In [11]:
# # page 크롤링
# page_links = driver.find_elements_by_css_selector("#contents > div.new_pagenate > a")
# page_add = page_links[1].get_attribute('href')
# print(page_add)

# 페이지 리스트 추출
page = 300

page_list = []
for i in tqdm(range(1,page+1)):
    page_list.append("https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=" + str(i) + "&jv_total_data=031||")

clear_output()
print('수집된 page수:', len(page_list),'개')
print(page_list[0])

수집된 page수: 300 개
https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=1&jv_total_data=031||


In [12]:
page_list

['https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=1&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=2&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=3&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=4&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=5&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=6&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=7&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=8&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=9&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=10&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=11&jv_total_data=031||',
 'https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=12

In [13]:
# column 내용 생략 없이 출력
pd.set_option('display.max_colwidth', -1)

In [14]:
con_url = pd.DataFrame({'page_url':page_list})
con_url.head()

,page_url
0,https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=1&jv_total_data=031||
1,https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=2&jv_total_data=031||
2,https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=3&jv_total_data=031||
3,https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=4&jv_total_data=031||
4,https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=5&jv_total_data=031||


In [46]:
con_url.iloc[1]

page_url    https://www.xgolf.com/board/afternote/list.asp?pr_gotopage=2&jv_total_data=031||
Name: 1, dtype: object

In [16]:
page_nums = 5

title_list = [] 

options = webdriver.ChromeOptions()
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)

for k in range(page_nums):
    driver.get(con_url['page_url'][k])
    titles = "#contents > table > tbody > tr > td.title > div > a"
    title_raw = driver.find_elements_by_css_selector(titles)
    time.sleep(1)
    
    for j in tqdm(title_raw):
        title = j.text
        title_list.append(title)
    
    # time.sleep(1)
    # driver.close()

clear_output()    
print('title갯수: ', len(title_list))
    

title갯수:  2500


In [17]:
print('title갯수:',len(title_list),'개 / ', len(title_list)/500,'페이지 수집')


title갯수: 2500 개 /  5.0 페이지 수집


In [19]:

url_list = []

options = webdriver.ChromeOptions()
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)

for l in range(page_nums):
    # options = webdriver.ChromeOptions()
    # chrome_path = chromedriver_autoinstaller.install()
    # driver = webdriver.Chrome(chrome_path, options=options)
    driver.get(con_url['page_url'][l])
    url = "#contents > table > tbody > tr > td.title > div > a"
    links = driver.find_elements_by_css_selector(url)
    time.sleep(1)
    
    for l in tqdm(links):
        url_list.append(l.get_attribute('href'))
    
    # time.sleep(1)
    # driver.close()
    
clear_output() 
print('title갯수: ', len(url_list))

title갯수:  25000


In [20]:
print('title갯수:',len(title_list),'개 / ', len(title_list)/500,'페이지 수집')
print('title갯수:',len(url_list),'개 / ', len(url_list)/500,'페이지 수집')



title갯수: 25000 개 /  50.0 페이지 수집
title갯수: 25000 개 /  50.0 페이지 수집


In [ ]:
# 수집된 제목명 확인
# title_list

In [ ]:
# 수집된 제목명 확인
# url_list

In [21]:
# 수집된 url_list, title_list로 판다스 데이터프레임 만들기
xgolf_df = pd.DataFrame({'title':title_list, 'url':url_list})
xgolf_df

,title,url
0,굿 금강,https://www.xgolf.com/board/afternote/view.asp...
1,"양프로,티칭프로테스트를 앞둔 예비프",https://www.xgolf.com/board/afternote/view.asp...
2,경기북부의 몽베르,https://www.xgolf.com/board/afternote/view.asp...
3,재촉만 하지 않았으면 최고였을텐데...,https://www.xgolf.com/board/afternote/view.asp...
4,몽베르라운딩후기,https://www.xgolf.com/board/afternote/view.asp...
...,...,...
24995,최악의 골프장,https://www.xgolf.com/board/afternote/view.asp...
24996,이슬비 오는 안성 CC에서 라운드 - 좋아요,https://www.xgolf.com/board/afternote/view.asp...
24997,황당한 캐디를 만나다,https://www.xgolf.com/board/afternote/view.asp...
24998,잘 관리된 안성CC에서 지인들과의 좋은 란딩,https://www.xgolf.com/board/afternote/view.asp...


In [ ]:
# 저장하기
xgolf_df.to_csv("./data/blog_url.csv", encoding='utf-8-sig')
# df.to_csv("blog_url.csv", encoding='utf-8-sig')

### 2. 후기 내용 크롤링하기

In [18]:
import sys
import os
import pandas as pd
import numpy as np

In [38]:
# "url_list.csv" 불러오기
url_load = pd.read_csv("./data/blog_url.csv")
url_load = url_load.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

num_list = len(url_load)

print(num_list)
url_load

25000


,title,url
0,굿 금강,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996362&pr_gotopage=1&jv_total_data=031||
1,"양프로,티칭프로테스트를 앞둔 예비프",https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996348&pr_gotopage=1&jv_total_data=031||
2,경기북부의 몽베르,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996250&pr_gotopage=1&jv_total_data=031||
3,재촉만 하지 않았으면 최고였을텐데...,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16995889&pr_gotopage=1&jv_total_data=031||
4,몽베르라운딩후기,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16994496&pr_gotopage=1&jv_total_data=031||
...,...,...
24995,최악의 골프장,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757781&pr_gotopage=50&jv_total_data=031||
24996,이슬비 오는 안성 CC에서 라운드 - 좋아요,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757767&pr_gotopage=50&jv_total_data=031||
24997,황당한 캐디를 만나다,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757722&pr_gotopage=50&jv_total_data=031||
24998,잘 관리된 안성CC에서 지인들과의 좋은 란딩,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757665&pr_gotopage=50&jv_total_data=031||


In [39]:
# url_load = url_load.drop(url_load.index[0:301])
url_load['url']

0        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996362&pr_gotopage=1&jv_total_data=031||
1        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996348&pr_gotopage=1&jv_total_data=031||
2        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996250&pr_gotopage=1&jv_total_data=031||
3        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16995889&pr_gotopage=1&jv_total_data=031||
4        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16994496&pr_gotopage=1&jv_total_data=031||
                                                          ...                                                   
24995    https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757781&pr_gotopage=50&jv_total_data=031||
24996    https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757767&pr_gotopage=50&jv_total_data=031||
24997    https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757722&pr_gotopage=50&jv_

#### 이용후기 내용 수집 test

In [23]:
# 글 하나 띄우기
i = 1
url = url_load['url'][i]
driver = webdriver.Chrome(chrome_path, options=options)
driver.get(url)   # 글 띄우기

In [24]:
dict = {}
target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

In [25]:
# 제목 크롤링 
overlays = "#epilogue-view > div.header > h2"                                 
tit = driver.find_element_by_css_selector(overlays)         
title = tit.text

# 골프장명 크롤링 
overlays = "#epilogue-view > div.header > dl.info.clear > dd.golflink"                                 
golf_na = driver.find_element_by_css_selector(overlays)         
golf_name = golf_na.text

# 글쓴이 크롤링 
overlays = "#epilogue-view > div.header > dl.info.clear > dd.writer"                                 
nick = driver.find_element_by_css_selector(overlays)         
nickname = nick.text

# 날짜 크롤링
overlays = "#epilogue-view > div.header > dl.info.clear > dd.date"                              
date = driver.find_element_by_css_selector(overlays)         
datetime = date.text

# 골프장 스코어 크롤링
overlays = "#epilogue-view > div.header > dl.grade.clear > dd.avg > ul > li > span"                              
golf_score = driver.find_element_by_css_selector(overlays)      
golfscore = golf_score.text

# 골프장 스코어 크롤링
overlays = "#contents > div.club_info_area > div.category_point > dl.first > dd"                              
caddie_score = driver.find_element_by_css_selector(overlays)         
caddiescore = caddie_score.text

# 후기내용 크롤링
overlays = "#board_content"                              
contents = driver.find_element_by_css_selector(overlays)         
contents = contents.text

# 항목별평점 크롤링(캐디서비스)
caddie_score = driver.find_element_by_xpath("""//*[@id="epilogue-view"]/div[1]/dl[2]/dd[2]/img""")   
caddie_score = caddie_score.get_attribute('src')
caddiescore = re.sub(r'[^0-9]', '', caddie_score)

# 항목별평점 크롤링(코스관리)
course_score = driver.find_element_by_xpath("""//*[@id="epilogue-view"]/div[1]/dl[2]/dd[3]/img""")   
course_score = course_score.get_attribute('src')
coursescore = re.sub(r'[^0-9]', '', course_score)

# 항목별평점 크롤링(가격만족도)
price_score = driver.find_element_by_xpath("""//*[@id="epilogue-view"]/div[1]/dl[2]/dd[4]/img""")   
price_score = price_score.get_attribute('src')
pricescore = re.sub(r'[^0-9]', '', price_score)

# 항목별평점 크롤링(부대시설/식사)
facil_score = driver.find_element_by_xpath("""//*[@id="epilogue-view"]/div[1]/dl[2]/dd[5]/img""")   
facil_score = facil_score.get_attribute('src')
facility = re.sub(r'[^0-9]', '', facil_score)

# # 항목별평점 크롤링(숙박)
# room_score = driver.find_element_by_xpath("""//*[@id="epilogue-view"]/div[1]/dl[2]/dd[6]/img""")   
# room_score = room_score.get_attribute('src')
# roomscore = re.sub(r'[^0-9]', '', room_score)

print('-제목:',title)
print('-작성자명:',nickname)
print('-작성일:',datetime)
print('-골프장명:',golf_name)
print('-골프장평점:',golfscore)
print('-항목별평점(캐디서비스):',caddiescore)
print('-항목별평점(코스관리):',coursescore)
print('-항목별평점(가격만족도):',pricescore)
print('-항목별평점(부대시설/식사):',facility)
# print('-항목별평점(숙박):',roomscore)
print('')
print('-이용후기:',contents)




-제목: 양프로,티칭프로테스트를 앞둔 예비프로와 함께
-작성자명: Wls******
-작성일: 2022.07.23 11:08
-골프장명: 비에이비스타퍼블릭
-골프장평점: 10.0
-항목별평점(캐디서비스): 10
-항목별평점(코스관리): 10
-항목별평점(가격만족도): 10
-항목별평점(부대시설/식사): 10

-이용후기: 라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다
모든 좋았으나한 가지 아쉬움은 그린이 조금 느렸는데요,그래도 대체적으로 만족했습니다 동반자 분 프로테스트 잘 받으시길 빌며...


In [26]:
# 리뷰 하나는 target_info라는 딕셔너리에 담기게 되고,
target_info['title'] = title
target_info['nickname'] = nickname
target_info['datetime'] = datetime
target_info['golf_name'] = golf_name
target_info['golfscore'] = golfscore
target_info['caddiescore'] = caddiescore
target_info['coursescore'] = coursescore
target_info['pricescore'] = pricescore
target_info['facility'] = facility
target_info['contents'] = contents

target_info

{'title': '양프로,티칭프로테스트를 앞둔 예비프로와 함께',
 'nickname': 'Wls******',
 'datetime': '2022.07.23 11:08',
 'golf_name': '비에이비스타퍼블릭',
 'golfscore': '10.0',
 'caddiescore': '10',
 'coursescore': '10',
 'pricescore': '10',
 'facility': '10',
 'contents': '라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한 가지 아쉬움은 그린이 조금 느렸는데요,그래도 대체적으로 만족했습니다 동반자 분 프로테스트 잘 받으시길 빌며...'}

In [27]:
dict[i] = target_info
dict

{1: {'title': '양프로,티칭프로테스트를 앞둔 예비프로와 함께',
  'nickname': 'Wls******',
  'datetime': '2022.07.23 11:08',
  'golf_name': '비에이비스타퍼블릭',
  'golfscore': '10.0',
  'caddiescore': '10',
  'coursescore': '10',
  'pricescore': '10',
  'facility': '10',
  'contents': '라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한 가지 아쉬움은 그린이 조금 느렸는데요,그래도 대체적으로 만족했습니다 동반자 분 프로테스트 잘 받으시길 빌며...'}}

In [28]:
# 판다스로 만들기
import pandas as pd

result_df = pd.DataFrame.from_dict(dict, 'index')

result_df

,title,nickname,datetime,golf_name,golfscore,caddiescore,coursescore,pricescore,facility,contents
1,"양프로,티칭프로테스트를 앞둔 예비프로와 함께",Wls******,2022.07.23 11:08,비에이비스타퍼블릭,10.0,10,10,10,10,"라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한 가지 아쉬움은 그린이 조금 느렸는데요,그래도 대체적으로 만족했습니다 동반자 분 프로테스트 잘 받으시길 빌며..."


In [32]:
# 저장하기
result_df.to_excel("./data/blog_content.xlsx", encoding='utf-8-sig')
time.sleep(3)

### 3. 저장된 url별 자료수집

In [40]:
# url_load = url_load.drop(url_load.index[0:301])
url_load['url']

0        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996362&pr_gotopage=1&jv_total_data=031||
1        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996348&pr_gotopage=1&jv_total_data=031||
2        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996250&pr_gotopage=1&jv_total_data=031||
3        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16995889&pr_gotopage=1&jv_total_data=031||
4        https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16994496&pr_gotopage=1&jv_total_data=031||
                                                          ...                                                   
24995    https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757781&pr_gotopage=50&jv_total_data=031||
24996    https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757767&pr_gotopage=50&jv_total_data=031||
24997    https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757722&pr_gotopage=50&jv_

In [57]:
number = 24999
list(range(number, 0, -1))

[24999,
 24998,
 24997,
 24996,
 24995,
 24994,
 24993,
 24992,
 24991,
 24990,
 24989,
 24988,
 24987,
 24986,
 24985,
 24984,
 24983,
 24982,
 24981,
 24980,
 24979,
 24978,
 24977,
 24976,
 24975,
 24974,
 24973,
 24972,
 24971,
 24970,
 24969,
 24968,
 24967,
 24966,
 24965,
 24964,
 24963,
 24962,
 24961,
 24960,
 24959,
 24958,
 24957,
 24956,
 24955,
 24954,
 24953,
 24952,
 24951,
 24950,
 24949,
 24948,
 24947,
 24946,
 24945,
 24944,
 24943,
 24942,
 24941,
 24940,
 24939,
 24938,
 24937,
 24936,
 24935,
 24934,
 24933,
 24932,
 24931,
 24930,
 24929,
 24928,
 24927,
 24926,
 24925,
 24924,
 24923,
 24922,
 24921,
 24920,
 24919,
 24918,
 24917,
 24916,
 24915,
 24914,
 24913,
 24912,
 24911,
 24910,
 24909,
 24908,
 24907,
 24906,
 24905,
 24904,
 24903,
 24902,
 24901,
 24900,
 24899,
 24898,
 24897,
 24896,
 24895,
 24894,
 24893,
 24892,
 24891,
 24890,
 24889,
 24888,
 24887,
 24886,
 24885,
 24884,
 24883,
 24882,
 24881,
 24880,
 24879,
 24878,
 24877,
 24876,
 24875,


In [66]:
rev_url_load = url_load.iloc[::-1].reset_index()

In [67]:
rev_url_load

,index,title,url
0,24999,고객과의 라운딩으로는 최적입니다,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757633&pr_gotopage=50&jv_total_data=031||
1,24998,잘 관리된 안성CC에서 지인들과의 좋은 란딩,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757665&pr_gotopage=50&jv_total_data=031||
2,24997,황당한 캐디를 만나다,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757722&pr_gotopage=50&jv_total_data=031||
3,24996,이슬비 오는 안성 CC에서 라운드 - 좋아요,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757767&pr_gotopage=50&jv_total_data=031||
4,24995,최악의 골프장,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=9757781&pr_gotopage=50&jv_total_data=031||
...,...,...,...
24995,4,몽베르라운딩후기,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16994496&pr_gotopage=1&jv_total_data=031||
24996,3,재촉만 하지 않았으면 최고였을텐데...,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16995889&pr_gotopage=1&jv_total_data=031||
24997,2,경기북부의 몽베르,https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996250&pr_gotopage=1&jv_total_data=031||
24998,1,"양프로,티칭프로테스트를 앞둔 예비프",https://www.xgolf.com/board/afternote/view.asp?pr_board_code=16996348&pr_gotopage=1&jv_total_data=031||


In [69]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇
target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

# 수집할 글 갯수 정하기
number = 25000

options = webdriver.ChromeOptions()
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)

# 수집한 url 돌면서 데이터 수집
for m in range(0, number):
    url = rev_url_load['url'][m]
    driver.get(url)  
    time.sleep(1)

    # 크롤링
    try :
        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        
        # 제목 크롤링 
        overlays = "#epilogue-view > div.header > h2"                                 
        tit = driver.find_element_by_css_selector(overlays)         
        title = tit.text

        # 골프장명 크롤링 
        overlays = "#epilogue-view > div.header > dl.info.clear > dd.golflink"                                 
        golf_na = driver.find_element_by_css_selector(overlays)         
        golf_name = golf_na.text

        # 글쓴이 크롤링 
        overlays = "#epilogue-view > div.header > dl.info.clear > dd.writer"                                 
        nick = driver.find_element_by_css_selector(overlays)         
        nickname = nick.text

        # 날짜 크롤링
        overlays = "#epilogue-view > div.header > dl.info.clear > dd.date"                              
        date = driver.find_element_by_css_selector(overlays)         
        datetime = date.text

        # 골프장 스코어 크롤링
        overlays = "#epilogue-view > div.header > dl.grade.clear > dd.avg > ul > li > span"                              
        golf_score = driver.find_element_by_css_selector(overlays)      
        golfscore = golf_score.text

        # 골프장 스코어 크롤링
        overlays = "#contents > div.club_info_area > div.category_point > dl.first > dd"                              
        caddie_score = driver.find_element_by_css_selector(overlays)         
        caddiescore = caddie_score.text

        # 후기내용 크롤링
        overlays = "#board_content"                              
        contents = driver.find_element_by_css_selector(overlays)         
        contents = contents.text

        # 항목별평점 크롤링(캐디서비스)
        caddie_score = driver.find_element_by_xpath("""//*[@id="epilogue-view"]/div[1]/dl[2]/dd[2]/img""")   
        caddie_score = caddie_score.get_attribute('src')
        caddiescore = re.sub(r'[^0-9]', '', caddie_score)

        # 항목별평점 크롤링(코스관리)
        course_score = driver.find_element_by_xpath("""//*[@id="epilogue-view"]/div[1]/dl[2]/dd[3]/img""")   
        course_score = course_score.get_attribute('src')
        coursescore = re.sub(r'[^0-9]', '', course_score)

        # 항목별평점 크롤링(가격만족도)
        price_score = driver.find_element_by_xpath("""//*[@id="epilogue-view"]/div[1]/dl[2]/dd[4]/img""")   
        price_score = price_score.get_attribute('src')
        pricescore = re.sub(r'[^0-9]', '', price_score)

        # 항목별평점 크롤링(부대시설/식사)
        facil_score = driver.find_element_by_xpath("""//*[@id="epilogue-view"]/div[1]/dl[2]/dd[5]/img""")   
        facil_score = facil_score.get_attribute('src')
        facility = re.sub(r'[^0-9]', '', facil_score)
        
        # 리뷰 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['golf_name'] = golf_name
        target_info['golfscore'] = golfscore
        target_info['caddiescore'] = caddiescore
        target_info['coursescore'] = coursescore
        target_info['pricescore'] = pricescore
        target_info['facility'] = facility
        target_info['contents'] = contents

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[m] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(m,'[글제목]',title,'[날자]',datetime, ' [골프장명]',golf_name,' [골프장점수]',golfscore)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        # driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        # driver.close()
        time.sleep(3)
        continue 
        
    # 판다스로 만들기
    import pandas as pd
    result_df = pd.DataFrame.from_dict(dict, 'index')
    # 저장하기
    result_df.to_csv("./data/blog_content_24999.csv", encoding='utf-8-sig')
    time.sleep(2)

clear_output()
print('수집한 글 갯수: ', len(dict),'개')


0 [글제목] 고객과의 라운딩으로는 최적입니다 [날자] 2018.06.01 10:22  [골프장명] 골프존카운티안성W  [골프장점수] 9.0
1 [글제목] 잘 관리된 안성CC에서 지인들과의 좋은 란딩 [날자] 2018.06.01 10:29  [골프장명] 안성  [골프장점수] 10.0
2 [글제목] 황당한 캐디를 만나다 [날자] 2018.06.01 10:37  [골프장명] 골드  [골프장점수] 10.0
3 [글제목] 이슬비 오는 안성 CC에서 라운드 - 좋아요 [날자] 2018.06.01 10:44  [골프장명] 안성  [골프장점수] 10.0
4 [글제목] 최악의 골프장 [날자] 2018.06.01 10:46  [골프장명] 은화삼  [골프장점수] 3.0
5 [글제목] 가까운 골프장 [날자] 2018.06.01 10:49  [골프장명] 캐슬렉스  [골프장점수] 5.0
6 [글제목] 항상 즐거운 양주 [날자] 2018.06.01 10:53  [골프장명] 양주  [골프장점수] 10.0
7 [글제목] 골드cc [날자] 2018.06.01 10:54  [골프장명] 골드  [골프장점수] 7.0
8 [글제목] 가성비 좋은 타이거 [날자] 2018.06.01 11:03  [골프장명] 타이거  [골프장점수] 10.0
9 [글제목] 거의 완벽한 골프장이지만.. [날자] 2018.06.01 11:13  [골프장명] 중부  [골프장점수] 9.5
10 [글제목] 즐거운 라운딩 이었습니다! [날자] 2018.06.01 11:20  [골프장명] 5/31 양지파인  [골프장점수] 10.0
11 [글제목] 간만에 써닝포인트 즐거운 라운딩 [날자] 2018.06.01 11:21  [골프장명] 써닝포인트  [골프장점수] 8.0
12 [글제목] 재미진 라운딩 [날자] 2018.06.01 11:38  [골프장명] 신안(P9)  [골프장점수] 10.0
13 [글제목] 굿샷 ! [날자] 2018.06.01 13:20  [골프장명] 스카이밸리  [골프장점수] 10.0
14 [글제목] 제대로된 골프장.. 

120 [글제목] 좋지도 나쁘지도 않은.. [날자] 2018.06.03 19:20  [골프장명] 뉴서울  [골프장점수] 6.0
121 [글제목] 가성비 괘안네 [날자] 2018.06.03 19:53  [골프장명] 양지파인  [골프장점수] 9.0
122 [글제목] 힘든하루 [날자] 2018.06.03 21:02  [골프장명] 88  [골프장점수] 5.5
123 [글제목] 오래되어서 더 좋은 안성cc [날자] 2018.06.03 21:04  [골프장명] 안성  [골프장점수] 9.0
124 [글제목] 그린 에어레이션 [날자] 2018.06.03 22:36  [골프장명] 일동레이크  [골프장점수] 8.5
125 [글제목] 언제나 만족스러운 골프장 [날자] 2018.06.03 22:38  [골프장명] 기흥  [골프장점수] 9.5
126 [글제목] 은화삼 [날자] 2018.06.03 23:26  [골프장명] 은화삼  [골프장점수] 7.0
127 [글제목] 즐거운 하루 [날자] 2018.06.04 00:51  [골프장명] 스카이밸리  [골프장점수] 8.0
128 [글제목] 최고의 그린 컨디션 [날자] 2018.06.04 06:28  [골프장명] 88  [골프장점수] 7.5
129 [글제목] 멋진 광릉CC입니다 [날자] 2018.06.04 07:06  [골프장명] 한림광릉  [골프장점수] 10.0
130 [글제목] 멋진 골프장입니다 [날자] 2018.06.04 07:08  [골프장명] 양지파인  [골프장점수] 10.0
131 [글제목] 그냥 그렇습니다 [날자] 2018.06.04 07:10  [골프장명] 에덴블루  [골프장점수] 6.5
132 [글제목] 즐거운... [날자] 2018.06.04 07:12  [골프장명] 레이크우드  [골프장점수] 8.5
133 [글제목] 가성비 최고 [날자] 2018.06.04 07:23  [골프장명] 파주  [골프장점수] 9.5
134 [글제목] 벙개라운딩 [날자] 2018.06.04 07:51  [골프장명] 블루헤런  [골프장점수] 10.

239 [글제목] 뉴스프링빌 [날자] 2018.06.05 13:28  [골프장명] 뉴스프링빌  [골프장점수] 10.0
240 [글제목] 즐건라운딩 [날자] 2018.06.05 13:29  [골프장명] 여주신라  [골프장점수] 10.0
241 [글제목] 즐건라운딩 [날자] 2018.06.05 13:30  [골프장명] 스카이밸리  [골프장점수] 10.0
242 [글제목] 화창한 날씨 [날자] 2018.06.05 13:32  [골프장명] 이포  [골프장점수] 10.0
243 [글제목] 가성비가 좋아요. [날자] 2018.06.05 13:32  [골프장명] 레이크힐스안성(P9)  [골프장점수] 8.5
244 [글제목] 후기 [날자] 2018.06.05 13:36  [골프장명] 이포  [골프장점수] 9.0
245 [글제목] 태광cc [날자] 2018.06.05 13:56  [골프장명] 태광  [골프장점수] 7.5
246 [글제목] 솔트베이cc 야간라운딩 [날자] 2018.06.05 14:45  [골프장명] 솔트베이  [골프장점수] 10.0
247 [글제목] 힐링되는 라운딩! [날자] 2018.06.05 14:54  [골프장명] 포천아도니스  [골프장점수] 10.0
248 [글제목] 가성비 최고 [날자] 2018.06.05 14:56  [골프장명] 한양파인(P9)  [골프장점수] 7.5
249 [글제목] 아침에 순식간에 한판 [날자] 2018.06.05 14:58  [골프장명] 포천아도니스  [골프장점수] 8.5
250 [글제목] 좋은 코스지요.. 가깝고. [날자] 2018.06.05 15:00  [골프장명] 태광  [골프장점수] 9.5
251 [글제목] 써닝포인트 후기 [날자] 2018.06.05 15:21  [골프장명] 써닝포인트  [골프장점수] 10.0
252 [글제목] 아도니스 [날자] 2018.06.05 15:24  [골프장명] 포천아도니스  [골프장점수] 10.0
253 [글제목] 믿고 가는 은화삼.. [날자] 2018.06.05 16:11  [골프장명] 은화

366 [글제목] 그린이 최고네요 [날자] 2018.06.07 10:18  [골프장명] 몽베르  [골프장점수] 10.0
367 [글제목] 좋은 골프장에서 즐거운 시간이었습니다 [날자] 2018.06.07 10:28  [골프장명] 기흥  [골프장점수] 10.0
368 [글제목] 시원한오후라운딩 [날자] 2018.06.07 10:34  [골프장명] 강남300  [골프장점수] 10.0
369 [글제목] 정말 좋은골프장입니다 [날자] 2018.06.07 10:44  [골프장명] 사우스스프링스  [골프장점수] 10.0
370 [글제목] 역시 명문 [날자] 2018.06.07 10:51  [골프장명] 뉴코리아  [골프장점수] 10.0
371 [글제목] 좋아요 [날자] 2018.06.07 11:13  [골프장명] 써닝포인트  [골프장점수] 8.0
372 [글제목] 즐거운 라운딩~ [날자] 2018.06.07 11:24  [골프장명] 포천아도니스  [골프장점수] 9.5
373 [글제목] 정말 좋았네요 [날자] 2018.06.07 11:53  [골프장명] 포천아도니스  [골프장점수] 10.0
374 [글제목] 잔디도 좋고 주변 풍경도 너무너무 좋았어요 [날자] 2018.06.07 12:44  [골프장명] 노스팜(구 파주프리스틴밸리)  [골프장점수] 9.5
375 [글제목] 첨 가봤는데 골프장 좋네요^^ [날자] 2018.06.07 13:00  [골프장명] 금강  [골프장점수] 8.5
376 [글제목] 즐거운라운딩 [날자] 2018.06.07 13:57  [골프장명] 기흥  [골프장점수] 10.0
377 [글제목] 정말좋았어요 [날자] 2018.06.07 14:00  [골프장명] 강남300  [골프장점수] 10.0
378 [글제목] 즐거웠습니다 [날자] 2018.06.07 14:03  [골프장명] 타이거  [골프장점수] 9.0
379 [글제목] 오랫만에 [날자] 2018.06.07 14:53  [골프장명] 양주  [골프장점수] 10.0
380 [글제목] 전반적로 너무 엉망였네요

486 [글제목] 앵두가 있는 라운딩 [날자] 2018.06.09 11:08  [골프장명] 신안(P9)  [골프장점수] 10.0
487 [글제목] 넘 좋았습니다 [날자] 2018.06.09 11:44  [골프장명] 화성(P9)  [골프장점수] 9.0
488 [글제목] 친구 머리 올리러... [날자] 2018.06.09 12:05  [골프장명] 골프존카운티안성H  [골프장점수] 10.0
489 [글제목] 멋진 골프장에서 좋은 라운딩을 기억하며 [날자] 2018.06.09 12:50  [골프장명] 360도  [골프장점수] 9.5
490 [글제목] 시원한 란딩 [날자] 2018.06.09 13:04  [골프장명] 한림광릉  [골프장점수] 4.0
491 [글제목] 금강 좋아요 [날자] 2018.06.09 13:39  [골프장명] 금강  [골프장점수] 10.0
492 [글제목] 뷰가 좋아요 [날자] 2018.06.09 14:57  [골프장명] 캐슬파인  [골프장점수] 7.5
493 [글제목] 굿굿굿 [날자] 2018.06.09 16:12  [골프장명] 은화삼  [골프장점수] 10.0
494 [글제목] 굿굿굿 [날자] 2018.06.09 16:15  [골프장명] 뉴서울  [골프장점수] 10.0
495 [글제목] 서북코스 깔끔 [날자] 2018.06.09 19:43  [골프장명] 기흥  [골프장점수] 10.0
496 [글제목] 가깝고 관리 잘 된, 그러나 비싼 게 부담스러운 골프장. [날자] 2018.06.09 21:31  [골프장명] 기흥  [골프장점수] 6.5
497 [글제목] 자유cc 15시근처 티오프 [날자] 2018.06.09 23:16  [골프장명] 자유  [골프장점수] 10.0
498 [글제목] 뉴서울 [날자] 2018.06.10 00:09  [골프장명] 뉴서울  [골프장점수] 10.0
499 [글제목] 써닝포인트 [날자] 2018.06.10 00:13  [골프장명] 써닝포인트  [골프장점수] 10.0
500 [글제목] 88 [날자] 2018.06.10 00:

...

8078 [글제목] 코리아cc 후반 야간란딩 [날자] 2018.12.04 22:34  [골프장명] 코리아  [골프장점수] 9.5
8079 [글제목] 아리지후기 [날자] 2018.12.05 00:00  [골프장명] 아리지  [골프장점수] 8.0
8080 [글제목] 좀어려운골프장 [날자] 2018.12.05 09:04  [골프장명] 양지파인  [골프장점수] 8.5
8081 [글제목] 올해 마지막 라운딩 [날자] 2018.12.05 09:04  [골프장명] 수원  [골프장점수] 9.0
8082 [글제목] 기흥 CC 아주 좋아요. [날자] 2018.12.05 09:04  [골프장명] 기흥  [골프장점수] 10.0
8083 [글제목] 생각보다 춥지 않은 날씨에 즐거운 라운딩이었습니다. [날자] 2018.12.05 09:05  [골프장명] 푸른솔포천  [골프장점수] 9.5
8084 [글제목] 안성베네스트롼딩 [날자] 2018.12.05 09:12  [골프장명] 안성베네스트  [골프장점수] 10.0
8085 [글제목] 가성비 좋은 라운딩~ [날자] 2018.12.05 09:14  [골프장명] 레이크힐스안성(P9)  [골프장점수] 6.0
8086 [글제목] 지인과롼딩 [날자] 2018.12.05 11:05  [골프장명] 88  [골프장점수] 7.0
8087 [글제목] 즐거운 라운딩 [날자] 2018.12.05 12:49  [골프장명] 88  [골프장점수] 7.5
8088 [글제목] 좋아요 [날자] 2018.12.05 15:23  [골프장명] 뉴서울  [골프장점수] 8.5
8089 [글제목] 어렵다 [날자] 2018.12.05 15:27  [골프장명] 포천힐스  [골프장점수] 10.0
8090 [글제목] 가까운 곳에서 란딩 [날자] 2018.12.05 16:35  [골프장명] 한림광릉  [골프장점수] 8.0
8091 [글제목] 가깝고 좋은 구장.. [날자] 2018.12.05 18:58  [골프장명] 88  [골프장점수] 10.0
8092 [글제목] 그린은 개판 [날자] 20

8198 [글제목] 황제골프 [날자] 2018.12.16 11:32  [골프장명] 레이크사이드  [골프장점수] 9.0
8199 [글제목] 바람없어 좋은날 [날자] 2018.12.16 15:17  [골프장명] 포천힐스  [골프장점수] 8.5
8200 [글제목] 눈 때문에 [날자] 2018.12.16 16:47  [골프장명] 강남300  [골프장점수] 6.0
8201 [글제목] 퍼블릭 중 베스트 3 [날자] 2018.12.16 19:53  [골프장명] 마에스트로  [골프장점수] 10.0
8202 [글제목] 아일랜드 [날자] 2018.12.17 00:56  [골프장명] 아일랜드  [골프장점수] 6.0
8203 [글제목] 비에비.. 비스타 정말 별로 예요 [날자] 2018.12.17 09:02  [골프장명] 비에이비스타  [골프장점수] 5.5
8204 [글제목] 금강은 진리! [날자] 2018.12.17 09:13  [골프장명] 금강  [골프장점수] 10.0
8205 [글제목] 강추위속 즐거운 라운딩 [날자] 2018.12.17 09:44  [골프장명] 사우스스프링스  [골프장점수] 10.0
8206 [글제목] 엄청 추운날 기분 좋은 라운딩 입니다 [날자] 2018.12.17 09:55  [골프장명] 금강  [골프장점수] 8.0
8207 [글제목] 역쉬~~~~ [날자] 2018.12.17 10:05  [골프장명] 자유  [골프장점수] 10.0
8208 [글제목] 눈............... [날자] 2018.12.17 10:20  [골프장명] 솔모로  [골프장점수] 6.5
8209 [글제목] 친구들과 즐거운 라운드 [날자] 2018.12.17 11:41  [골프장명] 금강  [골프장점수] 7.5
8210 [글제목] 88 [날자] 2018.12.17 15:07  [골프장명] 88  [골프장점수] 10.0
8211 [글제목] 솔모로후기 [날자] 2018.12.17 16:03  [골프장명] 솔모로  [골프장점수] 7.0
8212 [글제목] 솔모로 [날자] 2018.12.

8317 [글제목] 거리 빼고는 나머지는 [날자] 2018.12.24 16:55  [골프장명] 강남300  [골프장점수] 7.0
8318 [글제목] 골프장 이용후기 [날자] 2018.12.24 17:09  [골프장명] 화성상록  [골프장점수] 8.5
8319 [글제목] 여주cc [날자] 2018.12.24 19:01  [골프장명] 여주  [골프장점수] 9.0
8320 [글제목] 뉴서울 [날자] 2018.12.24 19:31  [골프장명] 뉴서울  [골프장점수] 10.0
8321 [글제목] 골프장 가본곳 중 ？？최고 [날자] 2018.12.25 08:24  [골프장명] 블루헤런  [골프장점수] 9.5
8322 [글제목] 아주 잘 관리된 명문골프장 [날자] 2018.12.25 08:50  [골프장명] 레이크사이드  [골프장점수] 8.5
8323 [글제목] 솔모로 [날자] 2018.12.25 09:05  [골프장명] 솔모로  [골프장점수] 9.5
8324 [글제목] 깔끔해요 [날자] 2018.12.25 09:27  [골프장명] 아일랜드  [골프장점수] 10.0
8325 [글제목] 그린이튀어요 [날자] 2018.12.25 09:54  [골프장명] 강남300  [골프장점수] 6.0
8326 [글제목] 행복한 하루 [날자] 2018.12.25 11:58  [골프장명] 레이크사이드  [골프장점수] 10.0
8327 [글제목] 저렴한 그린피 [날자] 2018.12.25 13:37  [골프장명] 용인  [골프장점수] 9.5
8328 [글제목] 올해 마지막 골프 [날자] 2018.12.25 16:36  [골프장명] 기흥  [골프장점수] 10.0
8329 [글제목] 겨울골프 [날자] 2018.12.25 19:31  [골프장명] 솔모로  [골프장점수] 10.0
8330 [글제목] 당구장인가요? 탁구장인가요? 궁금합니다 [날자] 2018.12.25 20:32  [골프장명] 크리스탈밸리  [골프장점수] 3.5
8331 [글제목] 자유로골프장 good^^ [날자] 2018.12.25 2

8434 [글제목] 즐거운 라운딩이었습니다. [날자] 2019.01.12 09:15  [골프장명] 양평TPC  [골프장점수] 9.0
8435 [글제목] 뉴서울 굿 [날자] 2019.01.12 09:59  [골프장명] 뉴서울  [골프장점수] 8.0
8436 [글제목] 진심 끝판왕 [날자] 2019.01.12 10:00  [골프장명] 푸른솔포천  [골프장점수] 10.0
8437 [글제목] 날씨가 영상6도 [날자] 2019.01.12 10:04  [골프장명] 양평TPC  [골프장점수] 10.0
8438 [글제목] 넘 좋았습니다 [날자] 2019.01.12 17:23  [골프장명] 은화삼  [골프장점수] 10.0
8439 [글제목] 안성 너무 좋아요. [날자] 2019.01.12 18:36  [골프장명] 안성베네스트  [골프장점수] 10.0
8440 [글제목] 강남300 이용후기 [날자] 2019.01.12 19:20  [골프장명] 강남300  [골프장점수] 8.5
8441 [글제목] 겨울의 꿀 라운딩 [날자] 2019.01.12 22:52  [골프장명] 88  [골프장점수] 9.5
8442 [글제목] 가격짱 [날자] 2019.01.13 07:51  [골프장명] 솔모로  [골프장점수] 9.0
8443 [글제목] 겨울이라 그린은 실망 [날자] 2019.01.13 09:02  [골프장명] 강남300  [골프장점수] 6.5
8444 [글제목] 겨울 골프 [날자] 2019.01.13 09:03  [골프장명] 양평TPC  [골프장점수] 9.0
8445 [글제목] 싼맞에 가요! 가성비좋아요! [날자] 2019.01.13 09:05  [골프장명] 양지파인  [골프장점수] 7.5
8446 [글제목] 썬힐 파인,힐코스 그렇게 좁지않아요~ [날자] 2019.01.13 09:19  [골프장명] 썬힐  [골프장점수] 8.0
8447 [글제목] 겨울 바람 없는 포근한 라운딩 [날자] 2019.01.13 09:24  [골프장명] 한림광릉  [골프장점수] 8.0
8448 [글제목] 솔모로cc 

MaxRetryError: HTTPConnectionPool(host='localhost', port=51534): Max retries exceeded with url: /session/f185d29e6206412cebbf794677f22d82/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1684066a0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [39]:
result_df

,title,nickname,datetime,golf_name,golfscore,caddiescore,coursescore,pricescore,facility,contents
0,굿 금강,yun****,2022.07.23 11:13,금강,10.0,10,10,10,10,페어웨이 그린 베스트 명품구장
1,"양프로,티칭프로테스트를 앞둔 예비프로와 함께",Wls******,2022.07.23 11:08,비에이비스타퍼블릭,10.0,10,10,10,10,라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한...
2,경기북부의 몽베르,NV2*******,2022.07.23 10:04,몽베르,10.0,10,10,10,10,편안하고 관리가 잘된 골프코스에서 라운딩했네요
3,재촉만 하지 않았으면 최고였을텐데...,bab***,2022.07.22 23:11,스카이밸리,10.0,10,10,10,10,아침부터 비가 온 것을 제외하고는 좋았습니다.\n잘 관리된 페어웨이와 그린 그리고 ...
4,몽베르라운딩후기,ljk******,2022.07.22 11:25,몽베르,10.0,10,10,10,10,첫홀은 비가와서 징행할까 말까 했었는데\n그래도 멀리왔는데 치고가자 고 일행이 적극...
...,...,...,...,...,...,...,...,...,...,...
103,가깝고 가깝고 가깝고,NV2*******,2022.06.27 12:58,강남300,8.5,10,8,6,10,그 외는 별로..그린 느리고 그린주변 누더기\n페어웨이는 괜춘한데\n라이트 어둡고
104,해솔리아 대표 님 제발 코스 한번 다녀보시고 관리 좀 잘해주세요.,NV6*******,2022.06.27 12:07,해솔리아,5.0,6,2,2,10,오랜만에 부부 동반으로 가까워서 운동하였음 그린 엉망 코스는 정말로 관리가 안되었습...
105,골버디 1차 회동,per******,2022.06.27 11:21,더크로스비,6.5,6,8,6,6,우리는 골프레슨학교에서 만난 친구모임입니다. 서로 골프수준도 비슷하고 성향도 조화로...
106,역시 좋아요 그린관리 대박,skk**,2022.06.27 11:08,뉴서울,9.5,10,10,8,10,서울권에 2인플레이 가능한 곳 완전 강추
